In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import os
from tqdm import tqdm
import re
from PIL import Image

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

# 1) point this at your base folder, e.g. "/path/to/output_dir"
data_dir = "/home/mhs/research/thesis/balanced_img"

# 2) define any transforms you need (resize + to-tensor here)
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
])

# 3) load the full dataset; ImageFolder will assign class-indices based on subdir names
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# 4) compute split sizes
total_size = len(full_dataset)
train_size = int(0.70 * total_size)
val_size   = int(0.15 * total_size)
test_size  = total_size - train_size - val_size

# 5) do the split (set a seed for reproducibility)
train_ds, val_ds, test_ds = random_split(
    full_dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# 6) wrap in DataLoaders
batch_size = 32

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# now `train_loader`, `val_loader`, `test_loader` yield (images, labels) batches
print(f"Train: {len(train_ds)} images\nVal:   {len(val_ds)} images\nTest:  {len(test_ds)} images")


Train: 6367 images
Val:   1364 images
Test:  1365 images


In [3]:
train_ds[0][0].shape  # check the shape of a single image

torch.Size([3, 64, 64])

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import optuna
from optuna.pruners import MedianPruner

class OptunaCNN(nn.Module):
    def __init__(self, trial, num_classes=2, input_shape=None):
        super().__init__()
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()

        if input_shape is None:
            sample_batch, _ = next(iter(train_loader))
            input_shape = sample_batch.shape[1:]

        in_channels = input_shape[0]
        current_size = input_shape[1]  # Track the feature map size
        n_conv = trial.suggest_int('n_conv_layers', 1, 4)
        
        for i in range(n_conv):
            # Calculate maximum valid kernel size
            max_kernel = min(7, current_size)
            if max_kernel % 2 == 0:  # Ensure odd kernel size
                max_kernel -= 1
            
            # Only suggest kernel sizes that are valid
            if max_kernel < 3:
                break  # Stop adding layers if feature map is too small
                
            out_channels = trial.suggest_categorical(f"conv{i}_out", [16, 32, 64, 128])
            k = trial.suggest_int(f"conv{i}_k", 3, max_kernel, step=2)
            p = trial.suggest_int(f"conv{i}_pad", 0, k//2)
            
            self.convs.append(nn.Conv2d(in_channels, out_channels, kernel_size=k, padding=p))
            
            norm_type = trial.suggest_categorical(f"conv{i}_norm", ['none', 'batch', 'layer'])
            if norm_type == 'batch':
                self.norms.append(nn.BatchNorm2d(out_channels))
            elif norm_type == 'layer':
                self.norms.append(nn.GroupNorm(1, out_channels))
            else:
                self.norms.append(None)
                
            in_channels = out_channels
            # Update feature map size after conv and pool
            current_size = (current_size + 2*p - k + 1) // 2  # Account for pooling
            
        self.pool = nn.MaxPool2d(2)

        # Rest of initialization remains the same
        with torch.no_grad():
            x = torch.zeros(1, *input_shape)
            for conv, norm in zip(self.convs, self.norms):
                x = conv(x)
                if norm: x = norm(x)
                x = F.relu(x)
                x = self.pool(x)
            self.n_flatten = x.numel()

        self.fcs = nn.ModuleList()
        n_fc = trial.suggest_int('n_fc_layers', 1, 3)
        in_features = self.n_flatten
        for j in range(n_fc):
            out_feat = trial.suggest_int(f"fc{j}_units", 32, 512, step=32)
            self.fcs.append(nn.Linear(in_features, out_feat))
            in_features = out_feat
        self.output = nn.Linear(in_features, num_classes)

        self.dropout_p = trial.suggest_uniform('dropout', 0.0, 0.5)
        self.act_name = trial.suggest_categorical('activation', ['relu', 'leaky_relu', 'elu'])

    def forward(self, x):
        for conv, norm in zip(self.convs, self.norms):
            x = conv(x)
            if norm:
                x = norm(x)
            if self.act_name == 'relu':
                x = F.relu(x)
            elif self.act_name == 'leaky_relu':
                x = F.leaky_relu(x)
            elif self.act_name == 'elu':
                x = F.elu(x)
            x = self.pool(x)
        x = x.view(x.size(0), -1)
        for fc in self.fcs:
            x = fc(x)
            if self.act_name == 'relu':
                x = F.relu(x)
            elif self.act_name == 'leaky_relu':
                x = F.leaky_relu(x)
            elif self.act_name == 'elu':
                x = F.elu(x)
            x = F.dropout(x, p=self.dropout_p, training=self.training)
        return self.output(x)

def objective(trial):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    sample_batch, _ = next(iter(train_loader))
    input_shape = sample_batch.shape[1:]

    model = OptunaCNN(trial, num_classes=NUM_CLASSES, input_shape=input_shape).to(device)

    opt_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    if opt_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif opt_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    else:
        momentum = trial.suggest_uniform('momentum', 0.5, 0.9)
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    criterion = nn.CrossEntropyLoss()
    num_epochs = trial.suggest_int('epochs', 10, 50)
    patience = trial.suggest_int('patience', 3, 7)

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in tqdm(range(num_epochs), desc="Training Progress"):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            loss = criterion(model(images), labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        correct = total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_loss /= len(val_loader)
        val_acc = correct / total

        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                break

    return best_val_loss

NUM_CLASSES = len(full_dataset.classes)
study = optuna.create_study(direction='minimize', pruner=MedianPruner())
study.optimize(objective, n_trials=50, timeout=None)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, val in trial.params.items():
    print(f"    {key}: {val}")

# def get_img_type(img_path):
#     if "mri" in file_path:
#         return "mri"
#     elif "vg" in file_path:
#         return "vg"
#     else:
#         raise ValueError("Unknown file path. Cannot determine image type.")

# try:
#     import yaml
#     with open(f"best_params_{get_img_type(file_path)}.yaml", 'w') as f:
#         yaml.safe_dump(trial.params, f)
# except Exception as e:
#     print(f"Error saving parameters to YAML: {e}")
#     with open(f"best_params_{get_img_type(file_path)}.txt", 'w') as f:
#         for key, val in trial.params.items():
#             f.write(f"{key}: {val}\n")

# print("Best parameters saved.")

[I 2025-04-29 18:01:32,414] A new study created in memory with name: no-name-f84a622c-fd85-45f2-a76d-3225fdbbdea4
/tmp/ipykernel_146258/3697293579.py:76: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  self.dropout_p = trial.suggest_uniform('dropout', 0.0, 0.5)
/tmp/ipykernel_146258/3697293579.py:112: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/tmp/ipykernel_146258/3697293579.py:118: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  momentum = trial.suggest_uniform('momentum', 0.5, 0.9)
Training Progres

Best trial:
  Value: 1.0471430933752726
  Params: 
    n_conv_layers: 4
    conv0_out: 64
    conv0_k: 7
    conv0_pad: 3
    conv0_norm: batch
    conv1_out: 64
    conv1_k: 5
    conv1_pad: 1
    conv1_norm: layer
    conv2_out: 16
    conv2_k: 5
    conv2_pad: 0
    conv2_norm: batch
    conv3_out: 128
    conv3_k: 5
    conv3_pad: 2
    conv3_norm: batch
    n_fc_layers: 1
    fc0_units: 96
    dropout: 0.1352504254857248
    activation: relu
    optimizer: RMSprop
    lr: 0.0003554582958542614
    epochs: 15
    patience: 7


In [7]:
num_classes = NUM_CLASSES

In [9]:
import matplotlib.pyplot as plt

best_params = trial.params

class BestConfigCNN(nn.Module):
    def __init__(self, params, num_classes, input_shape):
        super().__init__()
        self.convs = nn.ModuleList()
        self.norms = nn.ModuleList()

        in_channels = input_shape[0]
        for i in range(params['n_conv_layers']):
            out_channels = params[f"conv{i}_out"]
            k = params[f"conv{i}_k"]
            p = params[f"conv{i}_pad"]
            self.convs.append(nn.Conv2d(in_channels, out_channels, kernel_size=k, padding=p))
            norm_type = params[f"conv{i}_norm"]
            if norm_type == 'batch':
                self.norms.append(nn.BatchNorm2d(out_channels))
            elif norm_type == 'layer':
                self.norms.append(nn.GroupNorm(1, out_channels))
            else:
                self.norms.append(None)
            in_channels = out_channels
        self.pool = nn.MaxPool2d(2)

        with torch.no_grad():
            x = torch.zeros(1, *input_shape)
            for conv, norm in zip(self.convs, self.norms):
                x = conv(x)
                if norm: x = norm(x)
                x = F.relu(x)
                x = self.pool(x)
            self.n_flatten = x.numel()

        self.fcs = nn.ModuleList()
        in_features = self.n_flatten
        for j in range(params['n_fc_layers']):
            out_feat = params[f"fc{j}_units"]
            self.fcs.append(nn.Linear(in_features, out_feat))
            in_features = out_feat
        self.output = nn.Linear(in_features, num_classes)

        self.dropout_p = params['dropout']
        self.act_name = params['activation']

    def forward(self, x):
        for conv, norm in zip(self.convs, self.norms):
            x = conv(x)
            if norm:
                x = norm(x)
            if self.act_name == 'relu':
                x = F.relu(x)
            elif self.act_name == 'leaky_relu':
                x = F.leaky_relu(x)
            elif self.act_name == 'elu':
                x = F.elu(x)
            x = self.pool(x)
        x = x.view(x.size(0), -1)
        for fc in self.fcs:
            x = fc(x)
            if self.act_name == 'relu':
                x = F.relu(x)
            elif self.act_name == 'leaky_relu':
                x = F.leaky_relu(x)
            elif self.act_name == 'elu':
                x = F.elu(x)
            x = F.dropout(x, p=self.dropout_p, training=self.training)
        return self.output(x)

sample_batch, _ = next(iter(train_loader))
input_shape = sample_batch.shape[1:]
model = BestConfigCNN(best_params, num_classes, input_shape).to(device)

if best_params['optimizer'] == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=best_params['lr'])
elif best_params['optimizer'] == 'RMSprop':
    optimizer = optim.RMSprop(model.parameters(), lr=best_params['lr'])
else:
    optimizer = optim.SGD(model.parameters(), lr=best_params['lr'], momentum=best_params.get('momentum', 0.9))

criterion = nn.CrossEntropyLoss()

num_epochs = 100

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train

    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val

    print(f"Epoch {epoch + 1}/{num_epochs}, "
          f"Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Validation Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%")

Epoch 1/100, Train Loss: 1.2801, Train Accuracy: 38.86%, Validation Loss: 1.1894, Validation Accuracy: 45.75%
Epoch 2/100, Train Loss: 1.1542, Train Accuracy: 46.63%, Validation Loss: 1.1372, Validation Accuracy: 47.36%
Epoch 3/100, Train Loss: 1.0828, Train Accuracy: 51.09%, Validation Loss: 1.0923, Validation Accuracy: 51.32%
Epoch 4/100, Train Loss: 1.0316, Train Accuracy: 54.23%, Validation Loss: 1.0961, Validation Accuracy: 50.81%
Epoch 5/100, Train Loss: 0.9651, Train Accuracy: 57.52%, Validation Loss: 1.1133, Validation Accuracy: 49.85%
Epoch 6/100, Train Loss: 0.8933, Train Accuracy: 61.49%, Validation Loss: 1.1795, Validation Accuracy: 48.46%
Epoch 7/100, Train Loss: 0.8175, Train Accuracy: 65.95%, Validation Loss: 1.0796, Validation Accuracy: 53.81%
Epoch 8/100, Train Loss: 0.7280, Train Accuracy: 70.30%, Validation Loss: 1.2934, Validation Accuracy: 50.66%
Epoch 9/100, Train Loss: 0.6462, Train Accuracy: 74.52%, Validation Loss: 1.2043, Validation Accuracy: 51.98%
Epoch 10/1